<a href="https://colab.research.google.com/github/Jeiderbeltran/IA_Tizon_Tardio_Hojas_Tomate_/blob/main/IA_Tizon_Tardio_Hojas_Tomate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
# Instalar dependencias
!pip install -q tensorflow tensorflow-datasets

In [16]:
# Clonar el repositorio
!pip install -q tensorflo
from google.colab import output
!git clone https://github.com/Jeiderbeltran/IA_Tizon_Tardio_Hojas_Tomate_.git

Cloning into 'IA_Tizon_Tardio_Hojas_Tomate_'...
remote: Enumerating objects: 3511, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 3511 (delta 0), reused 2 (delta 0), pack-reused 3508 (from 2)
Receiving objects: 100% (3511/3511), 117.16 MiB | 21.75 MiB/s, done.
Resolving deltas: 100% (7/7), done.
Updating files: 100% (7018/7018), done.
Encountered 1 file(s) that should have been pointers, but weren't:
	modelos_entrenados/modelo_tizon_v20251016_2051.keras


In [17]:

import os
base_dir = "/content/IA_Tizon_Tardio_Hojas_Tomate_/dataset_organizado"

In [18]:
import tensorflow as tf

BATCH_SIZE = 25
SEED = 123

train_dir = os.path.join(base_dir, "train")
val_dir   = os.path.join(base_dir, "val")
test_dir  = os.path.join(base_dir, "test")

train_ds = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    labels='inferred',
    label_mode='int',
    batch_size=None,
    shuffle=True,
    seed=SEED
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    val_dir,
    labels='inferred',
    label_mode='int',
    batch_size=None,
    shuffle=False
)

test_ds = tf.keras.utils.image_dataset_from_directory(
    test_dir,
    labels='inferred',
    label_mode='int',
    batch_size=None,
    shuffle=False
)


Found 2464 files belonging to 2 classes.
Found 700 files belonging to 2 classes.
Found 351 files belonging to 2 classes.


In [19]:
# Proceso de segmentacion
import numpy as np
import cv2
import matplotlib.pyplot as plt

def segment_leaf(image):
    import numpy as np
    import cv2

    img = np.array(image)

    #  Normalizar siempre la forma del array
    if img.ndim == 2:
        # Escala de grises → 3 canales RGB
        img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
    elif img.ndim == 3:
        if img.shape[-1] == 1:
            img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
        elif img.shape[-1] == 4:
            img = cv2.cvtColor(img, cv2.COLOR_RGBA2RGB)
        elif img.shape[-1] != 3:
            # Si llega algo extraño, replicar canales manualmente
            img = np.repeat(img[..., :1], 3, axis=-1)
    else:
        # Si llega mal dimensionada, convertir a RGB plano
        img = np.stack([img]*3, axis=-1)

    # Conversión a HSV
    try:
        hsv = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
    except Exception as e:
        print(f"Error al convertir a HSV. Forma de imagen: {img.shape}")
        # Convertir a RGB por defecto si el HSV falla
        hsv = cv2.cvtColor(cv2.cvtColor(img, cv2.COLOR_BGR2RGB), cv2.COLOR_RGB2HSV)

    # --- Rango de color para hoja sana y enferma ---
    lower_green = np.array([25, 40, 40])
    upper_green = np.array([95, 255, 255])
    mask_green = cv2.inRange(hsv, lower_green, upper_green)

    lower_brown = np.array([10, 30, 30])
    upper_brown = np.array([25, 255, 255])
    mask_brown = cv2.inRange(hsv, lower_brown, upper_brown)

    mask = cv2.bitwise_or(mask_green, mask_brown)

    # --- Limpieza con morfología ---
    kernel = np.ones((5,5), np.uint8)
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)

    # --- Segmentar hoja más grande ---
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if contours:
        c = max(contours, key=cv2.contourArea)
        mask_clean = np.zeros_like(mask)
        cv2.drawContours(mask_clean, [c], -1, 255, -1)
        segmented = cv2.bitwise_and(img, img, mask=mask_clean)
    else:
        segmented = img

    return segmented



In [21]:
 # Proceso de Eliminacion de ruido

def denoise_image(image):
    denoised = cv2.medianBlur(image, 3)
    return denoised


In [22]:
 # Convertir a escala de grises
def to_grayscale(image):
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    return gray

In [23]:
  # Porceso de normalizacion
def normalize_image(image):
    norm = image.astype("float32") / 255.0
    return norm


In [24]:
# Aumento de datos
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal_and_vertical"),
    tf.keras.layers.RandomRotation(0.1),
    tf.keras.layers.RandomZoom(0.1),
    tf.keras.layers.RandomContrast(0.1),
])


In [25]:
def custom_preprocessing(image, label):
    import tensorflow as tf
    import numpy as np
    import cv2

    # Convertir a numpy
    if isinstance(image, tf.Tensor):
        image = image.numpy()

    # imagen tiene 3 canales
    if len(image.shape) == 2:  # Escala de grises
        image = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)
    elif image.shape[-1] == 1:
        image = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)
    elif image.shape[-1] == 4:  # Con alfa
        image = cv2.cvtColor(image, cv2.COLOR_RGBA2RGB)

    # --- Etapas del preprocesamiento ---
    image = segment_leaf(image)       # Segmentación por color HSV
    image = denoise_image(image)      # Eliminación de ruido
    image = to_grayscale(image)       # Conversión a gris
    image = cv2.resize(image, (256, 256))  # Escalado uniforme
    image = normalize_image(image)    # Normalización [0,1]

    # Expandir a (256,256,1)
    image = np.expand_dims(image, axis=-1)

    return image.astype("float32"), label


In [26]:
def tf_wrapper(image, label):
    img, lbl = tf.py_function(
        func=custom_preprocessing,
        inp=[image, label],
        Tout=[tf.float32, tf.int32]
    )

    # Quitar dimensiones sobrantes
    img = tf.squeeze(img)
    img = tf.reshape(img, (256, 256, 1))
    lbl.set_shape(())

    return img, lbl


In [27]:
# Aplicar al dataset
train_ds_pre = train_ds.map(tf_wrapper, num_parallel_calls=tf.data.AUTOTUNE).batch(BATCH_SIZE)
val_ds_pre   = val_ds.map(tf_wrapper, num_parallel_calls=tf.data.AUTOTUNE).batch(BATCH_SIZE)
test_ds_pre  = test_ds.map(tf_wrapper, num_parallel_calls=tf.data.AUTOTUNE).batch(BATCH_SIZE)

# Aumento de datos
train_ds_pre = train_ds_pre.map(lambda x, y: (data_augmentation(x, training=True), y))

# Prefetch
train_ds_pre = train_ds_pre.prefetch(tf.data.AUTOTUNE)
val_ds_pre   = val_ds_pre.prefetch(tf.data.AUTOTUNE)
test_ds_pre  = test_ds_pre.prefetch(tf.data.AUTOTUNE)

In [28]:
import tensorflow as tf
from tensorflow.keras import layers, models



def build_cnn(input_shape=(256, 256, 1), num_classes=2):
    model = models.Sequential([


        # Primera capa
        layers.Conv2D(32, (3,3), activation='relu', input_shape=input_shape, padding="same"),
        layers.MaxPooling2D((2,2)),

        # Segunda capa
        layers.Conv2D(64, (3,3), activation='relu', padding="same"),
        layers.MaxPooling2D((2,2)),

        # Tercera capa
        layers.Conv2D(128, (3,3), activation='relu', padding="same"),
        layers.MaxPooling2D((2,2)),


        # Evitar sobreajuste
        layers.Dropout(0.3),

        # Aplanado y capa densa
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.3),

        # Capa de salida
        layers.Dense(num_classes, activation='softmax')
    ])

    # Compilación del modelo
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )

    return model

# Crear el modelo
cnn_model = build_cnn(input_shape=(256, 256, 1), num_classes=2)



/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [30]:
# Callbacks
callbacks = [
    tf.keras.callbacks.ModelCheckpoint("best_model.h5", save_best_only=True, monitor="val_loss", mode="min"),
    tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True)
]

# Entrenamiento
history = cnn_model.fit(
    train_ds_pre,
    validation_data=val_ds_pre,
    epochs=1,
    callbacks=callbacks
)

99/99 ━━━━━━━━━━━━━━━━━━━━ 0s 4s/step - accuracy: 0.5309 - loss: 0.9427

99/99 ━━━━━━━━━━━━━━━━━━━━ 385s 4s/step - accuracy: 0.5313 - loss: 0.9405 - val_accuracy: 0.6100 - val_loss: 0.6265


In [31]:
import os
import datetime
from tensorflow.keras.models import load_model
from google.colab import files

nombre_modelo = f"modelo_tizon_v{datetime.datetime.now().strftime('%Y%m%d_%H%M')}.keras"


best_model_path = "best_model.h5"
if not os.path.exists(best_model_path):
    raise FileNotFoundError(" No se encontró el archivo 'best_model.h5'. Asegúrate de haber terminado el entrenamiento correctamente.")


modelo = load_model(best_model_path)
modelo.save(nombre_modelo)
print(f" Modelo guardado  como: {nombre_modelo}")

 Modelo guardado  como: modelo_tizon_v20251016_2353.keras


In [32]:
#,Descargar automáticamente el archivo
files.download(nombre_modelo)
print(" Descarga iniciada. Espera unos segundos para guardar el archivo en tu PC.")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

 Descarga iniciada. Espera unos segundos para guardar el archivo en tu PC.


In [11]:
from google.colab import files
from tensorflow.keras.models import load_model

# === 1. Subir el modelo desde tu PC ===
uploaded = files.upload()

# Detectar automáticamente el nombre del archivo subido
nombre_modelo = list(uploaded.keys())[0]
print(f"Archivo detectado: {nombre_modelo}")

# === 2. Cargar el modelo ===
modelo = load_model(nombre_modelo)
print("Modelo cargado correctamente")




Saving modelo_tizon_v20251016_2051.keras to modelo_tizon_v20251016_2051 (1).keras
Archivo detectado: modelo_tizon_v20251016_2051 (1).keras
Modelo cargado correctamente


/usr/local/lib/python3.12/dist-packages/keras/src/saving/saving_lib.py:802: UserWarning: Skipping variable loading for optimizer 'adam', because it has 22 variables whereas the saved optimizer has 2 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [33]:
# Evaluar el modelo con el conjunto de prueba
test_loss, test_acc = modelo.evaluate(test_ds_pre)
print(f"Precisión del modelo cargado: {test_acc*100:.2f}%")
print(f"Pérdida final: {test_loss:.4f}")



15/15 ━━━━━━━━━━━━━━━━━━━━ 15s 857ms/step - accuracy: 0.4274 - loss: 0.6285
Precisión del modelo cargado: 61.82%
Pérdida final: 0.6277


In [ ]:
#





#